In [1]:
from moviepy.editor import *

In [2]:
Truth = VideoFileClip('../media/big_buck_bunny_480p_surround-fix.avi')
font = "Lato-Bold"
w,h = 640,360
fontsize = w/20
bottom = h-fontsize*2-20

# вырезаем фрагмент
Clip = Truth.subclip(0, 5)
# яркость
Clip2 = Clip.fx(vfx.colorx, 0.7)  
# меняем размер
Clip2 = Clip2.resize((w,h))

print fontsize

32


In [4]:
title = ImageClip('../images/like.png').resize(width = 100)
title = title.set_duration(4).set_start(0).crossfadeout( 0.5)
                         
#высчитываем размер отступа снизу
theigth = h-title.size[1]-w/15
print theigth

# title = title.set_position(("center", theigth))

title = title.set_position( lambda t: ("center",
                                max(h/theigth,int(h-0.9*h*t+1))+theigth) ) 


# соединяем тексты и видео
screen_1 = CompositeVideoClip([Clip2, title])

# ipython_display(title, width=w)
ipython_display(screen_1, width=640, fps=24)

218


 99%|█████████▉| 120/121 [00:14<00:00,  8.20it/s]


In [6]:
import gizeh
import moviepy.editor as mpy

W,H = 100,100 # width, height, in pixels
duration = 5 # duration of the clip, in seconds

def make_frame(t):
    surface = gizeh.Surface(W,H)
    radius = W*(1+ (t*(duration-t))**2 )/6
    circle = gizeh.circle(radius, xy = (W/2,H/2), fill=(1,0,0))
    circle.draw(surface)
    return surface.get_npimage()

clip = mpy.VideoClip(make_frame, duration=duration)
ipython_display(clip, width=200, fps=24)
# clip.write_gif("circle.gif",fps=15, opt="OptimizePlus", fuzz=10)

ImportError: No module named gizeh

In [7]:
# моушен блюр
import numpy as np

def supersample(clip, d, nframes):
    """ Replaces each frame at time t by the mean of `nframes` equally spaced frames
        taken in the interval [t-d, t+d]. This results in motion blur."""
    def fl(gf, t):
        tt = np.linspace(t-d, t+d, nframes) 
        avg = np.mean(1.0*np.array([gf(t_) for t_ in tt]),axis=0)
        return avg.astype("uint8")
    return clip.fl(fl)

#usage:
some_videoclip = screen_1
new_clip = supersample(some_videoclip, d=0.05, nframes=5)
ipython_display(new_clip, width=w, fps=24)

# new_clip.write_gif("test.gif", fps=15)

 99%|█████████▉| 120/121 [01:34<00:00,  1.13it/s]


In [8]:
import numpy as np
from moviepy.editor import *
from moviepy.video.tools.segmenting import findObjects

# WE CREATE THE TEXT THAT IS GOING TO MOVE, WE CENTER IT.

screensize = (720,460)
txtClip = TextClip('Cool effect',color='white', font="Amiri-Bold",
                   kerning = 5, fontsize=100)
cvc = CompositeVideoClip( [txtClip.set_pos('center')],
                        size=screensize)

# THE NEXT FOUR FUNCTIONS DEFINE FOUR WAYS OF MOVING THE LETTERS


# helper function
rotMatrix = lambda a: np.array( [[np.cos(a),np.sin(a)], 
                                 [-np.sin(a),np.cos(a)]] )

def vortex(screenpos,i,nletters):
    d = lambda t : 1.0/(0.3+t**8) #damping
    a = i*np.pi/ nletters # angle of the movement
    v = rotMatrix(a).dot([-1,0])
    if i%2 : v[1] = -v[1]
    return lambda t: screenpos+400*d(t)*rotMatrix(0.5*d(t)*a).dot(v)
    
def cascade(screenpos,i,nletters):
    v = np.array([0,-1])
    d = lambda t : 1 if t<0 else abs(np.sinc(t)/(1+t**4))
    return lambda t: screenpos+v*400*d(t-0.15*i)

def arrive(screenpos,i,nletters):
    v = np.array([-1,0])
    d = lambda t : max(0, 3-3*t)
    return lambda t: screenpos-400*v*d(t-0.2*i)
    
def vortexout(screenpos,i,nletters):
    d = lambda t : max(0,t) #damping
    a = i*np.pi/ nletters # angle of the movement
    v = rotMatrix(a).dot([-1,0])
    if i%2 : v[1] = -v[1]
    return lambda t: screenpos+400*d(t-0.1*i)*rotMatrix(-0.2*d(t)*a).dot(v)



# WE USE THE PLUGIN findObjects TO LOCATE AND SEPARATE EACH LETTER

letters = findObjects(cvc) # a list of ImageClips


# WE ANIMATE THE LETTERS

def moveLetters(letters, funcpos):
    return [ letter.set_pos(funcpos(letter.screenpos,i,len(letters)))
              for i,letter in enumerate(letters)]

clips = [ CompositeVideoClip( moveLetters(letters,funcpos),
                              size = screensize).subclip(0,5)
          for funcpos in [vortex, cascade, arrive, vortexout] ]

# WE CONCATENATE EVERYTHING AND WRITE TO A FILE

final_clip = concatenate_videoclips(clips)
ipython_display(final_clip, width=w, fps=24)


100%|█████████▉| 480/481 [00:17<00:00, 27.91it/s]
